In [19]:
import tqdm
import requests
from bs4 import BeautifulSoup

path = "https://fontsinuse.com"
links = []

req = requests.get(path)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
fonts = soup.find_all('a', {'class': 'fiu-entity-use-link__label'})
for font in fonts:
    links.append(font.get('href'))

In [27]:
use_links = []
for link in links[55:]:
    for i in range(1,9):
        req = requests.get(path+link+'/'+str(i))
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.find_all('div', {'class': 'fiu-gallery-item__img-wrapper'})
        for article in articles:
            use_links.append(article.find('a').get('href'))

In [88]:
import numpy as np

typeface_dict = {}
fiu_data = []
idx = 0

for i, link in enumerate(use_links):
    if link == '/about':
        continue
    req = requests.get(path+link)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    typefaces = soup.find('ul', {'class': "fiu-sample-list fiu-use-meta-list fiu-use-meta-list--families fiu-use-meta-list--editable"})\
                    .find_all('img')
    
    type_list = []
    for typeface in typefaces:
        key = typeface.get('title')
        type_list.append(key)
        if typeface_dict.get(key) == None:
            value = typeface.get('src')
            typeface_dict[key] = value
    
    paras = soup.find_all('p')
    para_list = []
    for para in paras:
        para_list.append(para.text)
    text = ' '.join(para_list)
    
    fiu_data.append((type_list, text))

    if (i+1)%100 == 0:
        print(i+1)
    if (i+1)%1000 == 0:
        np.save('fiu_data%d.npy'%(i+1), fiu_data)
        np.save('typeface_dict%d.npy'%(i+1), typeface_dict)
        fiu_data = []
        typeface_dict = {}
        i+=1
        
np.save('fiu_data%d.npy'%(i+1), fiu_data)
np.save('typeface_dict%d.npy'%(i+1), typeface_dict)

6100
6200
6300
6500
6600
6700
6800
6900


In [87]:
len(use_links)

6763